In [21]:
from qonnx.core.modelwrapper import ModelWrapper
import os
onnx_dir = 'onnx_files/'
if not(os.path.exists(onnx_dir)):
    os.mkdir(onnx_dir)
    
model = ModelWrapper("multiply_model.onnx")

In [22]:
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants

model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

model.save(onnx_dir + "multiply_model_finn.onnx")

In [25]:
# change this if you have a different PYNQ board, see list above
from finn.util.basic import pynq_part_map
pynq_board = "KV260_SOM"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

In [26]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
model = ModelWrapper(onnx_dir+"/multiply_model_finn.onnx")
model = model.transform(ZynqBuild(platform = pynq_board, period_ns = target_clk_ns))

IndexError: list index (0) out of range